# Text generation

In [1]:
import numpy as np
import tensorflow as tf
import os
import warnings
warnings.filterwarnings('ignore')

*******************

# LOAD data. Preprocessing

In [2]:
path_to_file1 = 'af.txt'
text = open(path_to_file1, 'rb').read().decode(encoding='utf-8')
text = text + text
print('Length of text: {} characters'.format(len(text)))

Length of text: 1242284 characters


In [3]:
print(text[:207])

Вкус пищи знает тот, кто ест; ответить может лишь тот, кого спрашивают; сны видит лишь тот, кто спит; а для злодея наилучшим примером служит судья, сам достойный суда.

Вспыльчивый никогда не познает истину.


In [4]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

117 unique characters


In [5]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])
text_as_int, len(text_as_int), len(text)

(array([ 53,  91, 100, ...,   0,   0,   0]), 1242284, 1242284)

In [6]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

В
к
у
с
 


In [7]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Вкус пищи знает тот, кто ест; ответить может лишь тот, кого спрашивают; сны видит лишь тот, кто спит;'
' а для злодея наилучшим примером служит судья, сам достойный суда.\n\nВспыльчивый никогда не познает ис'
'тину.\n\nДаже разум глупца мирится с истиной.\n\nДобро хорошо тогда, когда это истинное добро.\n\nЕсли не и'
'справишь зло, оно удвоится.\n\nЖадность бездонна — в эту пропасть можно падать бесконечно.\n\nИстинно вел'
'икий не может быть жадным.\n\nКривыми путями не добраться барке до гавани.\n\nЛишь тот, кто честен и добр'


In [8]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [9]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Вкус пищи знает тот, кто ест; ответить может лишь тот, кого спрашивают; сны видит лишь тот, кто спит'
Target data: 'кус пищи знает тот, кто ест; ответить может лишь тот, кого спрашивают; сны видит лишь тот, кто спит;'


In [10]:
BATCH_SIZE = 64
embedding_dim = 128
rnn_units = 1024
vocab_size=len(vocab)
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

# Model

In [11]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [12]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 128)           14976     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          4722688   
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
lstm_2 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
lstm_3 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
dense (Dense)                (64, None, 117)           119925    
Total params: 30,035,701
Trainable params: 30,035,701
Non-trainable params: 0
____________________________________________

In [14]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 117) # (batch_size, sequence_length, vocab_size)


In [15]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [16]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 ' выказывает мелкую душу.\n\nЛучшая месть — забвение, оно похоронит врага в прахе его ничтожества.\n\nЛуч'
Next Char Predictions: 
 'cёкркууРШФ/vmиО«mФцРоНdгфXЖзIыжмЧва/\n[ Вdёkё«пнarмюХОzеБАйо71ЭЯр—akбА7Е9ях1дРf7лЦНЧkс-;5ОуПh…ЯпVМп\nМ'


In [17]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=192*5,
    mode='max',
    monitor='accuracy',
    save_best_only=True,
    save_weights_only=True)

In [18]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
opt = tf.keras.optimizers.Adam()

In [19]:
model.compile(optimizer=opt, loss=loss, metrics='accuracy')

# Training

In [20]:
EPOCHS = 80

In [21]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback], verbose=0)

# Text gen

In [22]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [29]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.85

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [31]:
text_ = generate_text(model, start_string=u"Не ")
print(text_)

Не для моих знания своих возможности имельшем домачно оказывается вредна. Напротив того, правда в общем. А — боже вас сохрани — не читайте до обеда советских газет. Пациенты, не читающие газет, чувствуют себя природу.

Никакое право и великая обязанность свою в отношении настоящего времени, кто наиболее современный человек не принадлежит двум типам людей: человек рисует свой собственный порядок. Так же облудайте опасно, лучший из миров.

Наука связывает наслаждение, как наслаждение творить.

Есть е
